In [1]:
import random
import numpy as np
import pandas as pd
import os
import sys
import inspect
import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

In [2]:
# set current notebook path
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [3]:
from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

from NER_Module import load_pretrained_model, BertNerModel, NerGritDataset, NerDataLoader

In [ ]:
# import nltk

# nltk.download('punkt')

## Training mode

In [4]:
## datapath 
data = "F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\NER_Module\\test\\data"
train_file = 'train_preprocess.txt'
validation_file ='valid_preprocess.txt'
test_file = 'test_preprocess_masked_label.txt'
model_path = 'F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\model'

In [5]:
train_data_path = os.path.join(data,train_file)
validation_data_path = os.path.join(data,validation_file)
test_data_path = os.path.join(data,test_file)

In [6]:
## load pretrained model
base_model,tokenizer = load_pretrained_model(num_labels=NerGritDataset.NUM_LABELS, is_cuda=True)

2021-03-14 16:55:05,081 [NER_Module.model.model] [INFO] Loading pretrained indobert model.....
Some weights of BertForWordClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-03-14 16:55:25,939 [NER_Module.model.model] [INFO] Initialised model with indo-bert!!!!


In [7]:
base_model.device

device(type='cuda', index=0)

In [8]:
# load data
train_dataset = NerGritDataset(train_data_path, tokenizer, lowercase=True)
valid_dataset = NerGritDataset(validation_data_path, tokenizer, lowercase=True)
test_dataset = NerGritDataset(test_data_path, tokenizer, lowercase=True)

train_loader = NerDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
valid_loader = NerDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  
test_loader = NerDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [9]:
NerGritDataset.NUM_LABELS

7

In [10]:
w2i, i2w = NerGritDataset.LABEL2INDEX, NerGritDataset.INDEX2LABEL
# print(w2i)
# print(i2w)

In [11]:
print(type(i2w))

<class 'dict'>


In [12]:
## model initialisation
bert_model = BertNerModel(num_classes=NerGritDataset.NUM_LABELS, i2w=i2w, exp_id='test_2', random_state=33, device='cuda')

2021-03-14 16:55:34,368 [NER_Module.model.model] [INFO] Loading pretrained indobert model.....
Some weights of BertForWordClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-03-14 16:55:42,718 [NER_Module.model.model] [INFO] Initialised model with indo-bert!!!!
2021-03-14 16:55:42,719 [NER_Module.model.model] [INFO] Device: cuda


In [13]:
bert_model.train(train_loader=train_loader, val_loader=valid_loader, model_dir=model_path, num_epochs=10, 
                 evaluate_every=2, early_stop=3, valid_criterion='F1')

2021-03-14 16:55:42,729 [NER_Module.model.model] [INFO] Starting Training.....
2021-03-14 16:55:42,730 [NER_Module.model.model] [INFO] Device is set at cuda
2021-03-14 16:55:42,730 [NER_Module.model.model] [INFO] The model device is cuda:0
(Epoch 1) TRAIN LOSS:1.4055 LR:0.00002000:  37%|██████████████                        | 39/105 [00:48<00:11,  5.54it/s]2021-03-14 16:56:31,220 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 1) TRAIN LOSS:1.2666 LR:0.00002000:  76%|████████████████████████████▉         | 80/105 [00:56<00:04,  5.37it/s]2021-03-14 16:56:39,481 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 1) TRAIN LOSS:1.2137 LR:0.00002000: 100%|█████████████████████████████████████| 105/105 [01:02<00:00,  1.68it/s]
2021-03-14 16:56:45,452 [NER_Module.model.model] [INFO] (Epoch 1) TRAIN LOSS:1.2137 ACC:0.85 F1:0.20 REC:0.17 PRE:0.25 LR:0.00002000
(Epoch 2) TRAIN LOSS:0.9953 LR:0.00002000:  14%|█████▍                                | 15/105 [00:43<00:43,  2.06it/s]2021-03-14 16:57:28,646 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 2) TRAIN LOSS:0.9717 LR:0.00002000:  53%|████████████████████▎                 | 56/105 [00:52<00:13,  3.53it/s]2021-03-14 16:57:38,035 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 2) TRAIN LOSS:0.9502 LR:0.00002000:  92%|███████████████████████████████████   | 97/105 [01:01<00:01,  5.23it/s]2021-03-14 16:57:47,079 [NER_Module.model.backbone] [INFO] Device is set to cuda


...

(Epoch 9) TRAIN LOSS:0.4564 LR:0.00002000:  70%|██████████████████████████▊           | 74/105 [00:55<00:05,  5.30it/s]2021-03-14 17:07:59,993 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 9) TRAIN LOSS:0.4484 LR:0.00002000: 100%|█████████████████████████████████████| 105/105 [01:02<00:00,  1.68it/s]
2021-03-14 17:08:07,088 [NER_Module.model.model] [INFO] (Epoch 9) TRAIN LOSS:0.4484 ACC:0.98 F1:0.90 REC:0.91 PRE:0.89 LR:0.00002000
(Epoch 10) TRAIN LOSS:0.4283 LR:0.00002000:   9%|███▎                                  | 9/105 [00:41<03:56,  2.46s/it]2021-03-14 17:08:48,328 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 10) TRAIN LOSS:0.4157 LR:0.00002000:  48%|█████████████████▌                   | 50/105 [00:49<00:11,  4.95it/s]2021-03-14 17:08:56,453 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 10) TRAIN LOSS:0.4130 LR:0.00002000:  87%|████████████████████████████████     | 91/105 [00:57<00:02,  5.13it/s]2021-03-14 17:09:04,736 [NER_Module.model.backbone] [INFO] Device is set to cuda


(Epoch 10) TRAIN LOSS:0.4059 LR:0.00002000: 100%|████████████████████████████████████| 105/105 [01:01<00:00,  1.71it/s]
2021-03-14 17:09:08,702 [NER_Module.model.model] [INFO] (Epoch 10) TRAIN LOSS:0.4059 ACC:0.99 F1:0.92 REC:0.92 PRE:0.91 LR:0.00002000
  0%|                                                                                           | 0/14 [00:00<?, ?it/s]2021-03-14 17:09:48,351 [NER_Module.model.backbone] [INFO] Device is set to cuda
VALID LOSS:0.5839 ACC:0.95 F1:0.70 REC:0.74 PRE:0.67:   0%|                                     | 0/14 [00:00<?, ?it/s]2021-03-14 17:09:48,417 [NER_Module.model.backbone] [INFO] Device is set to cuda
VALID LOSS:0.5350 ACC:0.96 F1:0.80 REC:0.85 PRE:0.76:  14%|████▏                        | 2/14 [00:00<00:00, 16.53it/s]2021-03-14 17:09:48,473 [NER_Module.model.backbone] [INFO] Device is set to cuda
VALID LOSS:0.5528 ACC:0.95 F1:0.73 REC:0.76 PRE:0.70:  14%|████▏                        | 2/14 [00:00<00:00, 16.53it/s]2021-03-14 17:09:48,541 [NE

In [14]:
result_df = bert_model.predict(test_loader=test_loader, save_path= model_path, filename='result_test_2')

2021-03-14 17:10:39,408 [NER_Module.model.model] [INFO] Prediction mode....
  7%|█████▉                                                                             | 1/14 [00:39<08:33, 39.53s/it]2021-03-14 17:11:18,946 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-14 17:11:19,020 [NER_Module.model.backbone] [INFO] Device is set to cuda
 21%|█████████████████▊                                                                 | 3/14 [00:39<05:04, 27.70s/it]2021-03-14 17:11:19,106 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-14 17:11:19,154 [NER_Module.model.backbone] [INFO] Device is set to cuda
 36%|█████████████████████████████▋                                                     | 5/14 [00:39<02:54, 19.40s/it]2021-03-14 17:11:19,209 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-14 17:11:19,263 [NER_Module.model.backbone] [INFO] Device is set to cuda
 50%|█████████████████████████████████████████▌                                  

## Load model and predict (No training)

In [17]:
## model_path 
model_folder = './'
filename = 'best_model_test_2.th'
model_path = os.path.join(model_folder,filename)

In [15]:
## model initialisation
bert_predict_model = BertNerModel(num_classes=NerGritDataset.NUM_LABELS, i2w=i2w, exp_id='test_2', random_state=33, device='cuda')

2021-03-15 10:46:46,854 [NER_Module.model.model] [INFO] Loading pretrained indobert model.....
Some weights of BertForWordClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-03-15 10:46:55,945 [NER_Module.model.model] [INFO] Initialised model with indo-bert!!!!
2021-03-15 10:46:55,945 [NER_Module.model.model] [INFO] Device: cuda


In [18]:
## load custom model
bert_predict_model, bert_predict_tokenizer=bert__predict_model.load_modellocal(path=model_path, load_tokenizer= True)

2021-03-15 10:50:49,793 [NER_Module.model.model] [INFO] Loading model from local path in cuda device
2021-03-15 10:50:50,229 [NER_Module.model.model] [INFO] Loading Tokenizer from indo-Bert


In [21]:
## load test_data
predict_dataset = NerGritDataset(test_data_path, bert_predict_tokenizer, lowercase=True)

In [22]:
## load data trainer
predict_loader = NerDataLoader(dataset=predict_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [19]:
bert_predict_model.device

device(type='cuda')

In [26]:
result_path ='F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\model'

In [27]:
predict_df = bert_predict_model.predict(test_loader=predict_loader, save_path= result_path, filename='predict_test_2')

2021-03-15 11:00:32,658 [NER_Module.model.model] [INFO] Prediction mode....
  7%|█████▉                                                                             | 1/14 [00:39<08:33, 39.48s/it]2021-03-15 11:01:12,145 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-15 11:01:12,225 [NER_Module.model.backbone] [INFO] Device is set to cuda
 21%|█████████████████▊                                                                 | 3/14 [00:39<05:04, 27.66s/it]2021-03-15 11:01:12,315 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-15 11:01:12,363 [NER_Module.model.backbone] [INFO] Device is set to cuda
 36%|█████████████████████████████▋                                                     | 5/14 [00:39<02:54, 19.38s/it]2021-03-15 11:01:12,420 [NER_Module.model.backbone] [INFO] Device is set to cuda
2021-03-15 11:01:12,474 [NER_Module.model.backbone] [INFO] Device is set to cuda
 50%|█████████████████████████████████████████▌                                  

In [28]:
## Compare two dataframes are equal 
predict_df.equals(result_df)

True